In [ ]:
import json
import pandas as pd
import numpy as np
from pymongo import MongoClient
import matplotlib.pyplot as plt
import seaborn as sns
import datetime

In [ ]:
client = MongoClient('mongodb://######################/')
db = client['SONAR_CHALLENGE']
collection = db["python_repositories_metadata"]

In [ ]:
def get_repos_with_metadata():
    
    df = pd.DataFrame(columns=['full_name','stargazers_count'])
    eol_date = datetime.datetime.strptime('2022-01-01T00:00:00-05:00', "%Y-%m-%dT%H:%M:%S%z")
    
    cursors = collection.find({'available':True,'revised':True})
        
    for repo in cursors:
        
        last_update = datetime.datetime.strptime(repo['pushed_at'], '%Y-%m-%dT%H:%M:%S%z')   
        
        if last_update >= eol_date and repo['language'] == 'Python':
            
            entry = pd.DataFrame.from_dict({ 
                'full_name': [repo['full_name']], 
                'stargazers_count': [repo['stargazers_count']]
            })

            df = pd.concat([df, entry], ignore_index=True)
        
    new_dtypes = {'stargazers_count': int}
    df = df.astype(new_dtypes)
    
    return df

In [ ]:
df_popularity = get_repos_with_metadata()

In [ ]:
df_popularity.describe()

In [ ]:
sns.boxplot(x = 'stargazers_count', data = df_popularity, showfliers = False)

In [ ]:
df_popularity_mask = df_popularity['stargazers_count'] >= 1513
filtered_df_popularity = df_popularity[df_popularity_mask]

In [ ]:
filtered_df_popularity

In [ ]:
def set_clone_syntax_status(dataframe):
    for index, row in dataframe.iterrows():
        filter_query = { 'full_name': row['full_name'] }
        set_clone_syntax_flag = { '$set': { 'clone': False, 'syntax': False } }

        collection.update_one(filter_query, set_clone_syntax_flag)

In [ ]:
# set_clone_syntax_status(filtered_df_popularity)